In [5]:
!python3.10 -m pip install guardrails-ai

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!python3.10 -m pip install urllib3

Defaulting to user installation because normal site-packages is not writeable


In [7]:
!guardrails hub install hub://guardrails/detect_pii

Installing hub://guardrails/detect_pii...
[====] Fetching manifestst
[=   ] Downloading dependenciespendencies
[ ===] Running post-install setuptall setup
✅Successfully installed guardrails/detect_pii version 0.0.5!


Import validator:
from guardrails.hub import DetectPII

Get more info:
https://hub.guardrailsai.com/validator/guardrails/detect_pii



In [8]:
!guardrails hub install hub://guardrails/teste --quiet

Installing hub://guardrails/teste...
ERROR:guardrails-cli:404
ERROR:guardrails-cli:Not Found
ERROR:guardrails-cli:Failed to install hub://guardrails/teste


In [8]:
!python3.10 -m spacy download pt_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 2.3 MB/s eta 0:00:002.3 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [13]:
from guardrails.validators import (register_validator, Validator, FailResult, ValidationResult, PassResult)
from guardrails import Guard, OnFailAction
from guardrails.hub import DetectPII
from typing import Any, Dict, Optional, Callable, List

from presidio_analyzer import AnalyzerEngine, PatternRecognizer, RecognizerResult, Pattern
import re

from transformers import BertTokenizer, BertModel
import torch
import os
import numpy as np
import datetime as dt

import unicodedata
import spacy
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/levi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
if os.path.isdir("./bert_tokenizer") and os.path.isdir("./bert_model"):
    tokenizer = BertTokenizer.from_pretrained("./bert_tokenizer")
    model = BertModel.from_pretrained("./bert_model")
else:
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = BertModel.from_pretrained("bert-base-uncased")
    
    tokenizer.save_pretrained("./bert_tokenizer")
    model.save_pretrained("./bert_model")

In [12]:
def remove_pointing(text:str) -> str:
    """
        Remove pontuação do texto 
        Args:
            text: O texto a ser tratado
        
        Returns:
            O texto sem pontuação
    """
    ans = ""
    for c in text:
          if unicodedata.category(c) != 'Po':
               ans+=c
    return ans 


def remove_accent(text:str) -> str:
    """
        Remove os acentos das palavras do texto
        Args:
            text: O texto a ser tratado
        
        Returns:
            O texto sem acentos
    """
    nfkd = unicodedata.normalize('NFKD', text)
    ans = ""
    for c in nfkd:
         if not unicodedata.combining(c):
              ans += c
    return ans


def lemmatize(text: str) -> str:
    """
        Aplica lematização no texto
        Args:
            text: O texto a ser tratado
        Returns:
            O texto lematizado
    """
    nlp = spacy.load('pt_core_news_sm')
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])
   
   
def remove_stopwords(text: str) -> str:  
        """
            Remove as palavras irrelevantes do texto
            Args:
                words:  texto a ser tratado
            
            Returns:
                Lista de string sem as palavras irrelevantes
        """
        text_without_pointing_and_accent = remove_accent(remove_pointing(text))
        lemmatized_text = lemmatize(text_without_pointing_and_accent)
        words_splitted = lemmatized_text.split()     
        stop_words = set(stopwords.words("portuguese"))
        relevant_text = ""
        for word in words_splitted:
            if word.lower() not in stop_words and len(word) > 2 and len(word) < 15:
                relevant_text += re.sub(r'[^a-zA-Z0-9]', '', word.lower()) + " "
                
        return relevant_text
 
 
def generate_embeddings(text: str) -> List[List]:
       
    """
    Retorna os embeddings do texto passado como parâmetro da função.
    returns: Embeddings das palavras.
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    with torch.no_grad():
        outputs = model(**inputs)
        
    embeddings = outputs.last_hidden_state
    sentence_embedding = embeddings.mean(dim=1)
    
    return sentence_embedding


def cosine_similarity(text1: str, text2: str) -> float:
    """
    Calcula a similaridae cosseno entre dois textos.
    Gera-se os embeddings para cada texto e em seguida, fazemos a similarida usando os embeddings. 
    returns: Grau de similaridade em um intervalo fechado entre 0 e 1.
    """
    emb_text1 = generate_embeddings(text1)
    emb_text2 = generate_embeddings(text2)
    
    emb_text1 = emb_text1.cpu().numpy()
    emb_text2 = emb_text2.cpu().numpy()
    
    dot_product = np.dot(emb_text1, emb_text2.T)
    text1Norm = np.linalg.norm(emb_text1)
    text2Norm = np.linalg.norm(emb_text2)
    
    return dot_product / (text1Norm * text2Norm)


def split_text(text, max_length: int = 10) -> List:
        
    """
    Divide o texto em alguns pedaços de frases.
    returns: Lista de frases.
    """
    
    tokens = tokenizer.tokenize(text)
    chunks = [tokenizer.convert_tokens_to_string(tokens[i:i + max_length]) for i in range(0, len(tokens), max_length)]
    
    return chunks

def similarity_between_texts(text1, text2) -> float:
        
    """
    Calcula a similaridade entre dois textos.
    returns: Similaridade cosseno.
    """
    text1 = remove_stopwords(text1)
    text2 = remove_stopwords(text2)
    
    print(text1)
    print(text2)
    
    split_text1 = split_text(text1)
    split_text2 = split_text(text2)
    
    similarities = []
    
    if len(split_text1) == 0 or len(split_text2) == 0:
        return 0
        
    for text1 in split_text1:
        for text2 in split_text2:
            similarities.append(cosine_similarity(text1, text2)[0][0])
            
    media = np.mean(similarities)
    print(media, similarities)
    return media
    

In [13]:
@register_validator(name="guardrails/teste", data_type="string")
class ValidadorDeSimilaridade(Validator):
    def __init__(self, texto1: str, texto2: str, match_type: Optional[str] = None, on_fail: Optional[Callable] = None):        
        super().__init__(on_fail=on_fail, match_type=match_type)
        
        self.texto1 = texto1
        self.texto2 = texto2
        
    def validate(self, value: Any, metadata: Dict = {}) -> ValidationResult:
        similarity = float(f"{similarity_between_texts(self.texto1, self.texto2):.1f}")
        print(similarity)
        
        ideal_similarity = 0.7
        if similarity < ideal_similarity:
            print(f"{value}: Similaridade baixa {similarity} (menor que {ideal_similarity})")
            return FailResult(error_message="Erro")
        
        print(f"{value}: Similaridade alta de {similarity} (igual ou acima de {ideal_similarity})")
        return PassResult()

In [14]:
def teste(texto1, texto2):
    guard = Guard().use(
        ValidadorDeSimilaridade(texto1=texto1, texto2=texto2)
    )

    try:
        guard.parse("Agente Inteligente").model_validate
        print("Passou no teste de similaridade cosseno!")
    except Exception as e:
        print("Ocorreu um erro: ", e)

In [15]:
texto1 = """
lula 
"""

texto2 = """
carro
"""

teste(texto1=texto1, texto2=texto2)

/home/levi/.local/lib/python3.10/site-packages/guardrails/validator_service/__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


lula 
carro 
0.7625983 [0.7625983]
0.8
Agente Inteligente: Similaridade alta de 0.8 (igual ou acima de 0.7)
Passou no teste de similaridade cosseno!


In [16]:
texto1 = """
de onde sao os alunos de ciencia da computacao? liste todos os estados
"""

texto2 = """
Os alunos de Ciência da Computação vêm de diversos estados,
incluindo:


Pernambuco (PE)
Rio de Janeiro (RJ)
São Paulo (SP)
Minas Gerais (MG)
Bahia (BA)


Esses sao alguns exemplos dos estados de origem dos alunos."""

teste(texto1=texto1, texto2=texto2)

onde sao aluno ciencia computacao liste todo estado 
aluno ciencia computacao vir diverso estado incluir pernambuco rio janeiro sao paulo minas gerais bahia sao algum exemplo estado origem aluno 
0.659812 [0.8117717, 0.74846905, 0.5817227, 0.6511542, 0.5596056, 0.6946472, 0.6640316, 0.5304727, 0.8176472, 0.5385981]
0.7
Agente Inteligente: Similaridade alta de 0.7 (igual ou acima de 0.7)
Passou no teste de similaridade cosseno!


In [17]:
texto1 = """
está um resumo das informações relevantes sobre os alunos do curso de Ciência da Computação
"""

texto2 = """
### Alunas (Feminino)

- **Quantidade Total**: 183
- **Estado Civil**
 - Solteiro: 175
 - Casado:3
- **Nacionalidade**: 100% Brasileira
- **Principais Estados**
 -PB:139
 - PE:17
- **Idade**:
 - Média: 21.68 anos
 - Mínima: 17 anos
 - Máxima: 39 anos
- **Política Afirmativa**:
 - Bon. estadual: 40
 - L2: 26
- **Cor/Raça**:
  -Branca: 92
  -Parda: 83
- **Renda Per Capita**:
 - Média: 6.78
- **Tipo de Ensino Médio**
 - Somente escola pública: 93
 - Somente escola privada:89

### Alunos (Masculino)

⁃ **Quantidade Total**: 658
- **Estado Civil**:
⁃ Solteiro: 641
 - Casado:6
- **Nacionalidade**: Predominantemente Brasileira (657 brasileiros, 1 estrangeiro)
- **Principais Estados**:
 -PB:498
 -PE: 47
⁃ **Idade**:
 ⁃ Média: 21.87 anos
 - Minima: 17 anos
 ⁃ Máxima: 43 anos
- **Política Afirmativa**
 - Bon. estadual: 127
 - L2:71
- **Cor/Raca**:
 ⁃ Branca: 337
 -Parda: 271
- **Renda Per Capita**:
 - Média: 7.05
- **Tipo de Ensino Médio**:
 ⁃ Somente escola pública: 329
- Somente escola privada: 327

Essas informações destacam a distribuição de gênero, origem
geográfica, política afirmativa, e outros aspectos demograficos e
educacionais dos alunos.
"""

teste(texto1, texto2)

resumo informacoes relevante sobre aluno curso ciencia computacao 
alunas feminino quantidade total 183 estado civil solteiro 175 casado3 nacionalidade 100 brasileira principais estado pb139 pe17 idade media 2168 ano minima ano maxima ano politica afirmativa bon estadual corraca branca parda renda per capita media 678 tipo ensino medio somente escola publicar somente escola privada89 alunos masculino quantidade total 658 estado civil solteiro 641 casado6 nacionalidade brasileira 657 brasileiro estrangeiro principais estado pb498 pe idade media 2187 ano minima ano maxima ano politica afirmativa bon estadual 127 l271 corraca branca 337 parda 271 renda per capita media 705 tipo ensino medio somente escola publicar 329 somente escola privado 327 informacoes destacar distribuicao genero origem geografico politico afirmativo outro aspecto demografico educacional aluno 
0.6869465 [0.7530056, 0.6495437, 0.60388887, 0.58420134, 0.5304056, 0.79059666, 0.63076955, 0.52720684, 0.73523057, 0.765186

In [18]:
texto1 = "Lula é o presidente do Brasil"
texto2 = "Lula não é o presidente do Brasil"

teste(texto1=texto1, texto2=texto2)

lula presidente brasil 
lula nao presidente brasil 
0.9101971 [0.9101971]
0.9
Agente Inteligente: Similaridade alta de 0.9 (igual ou acima de 0.7)
Passou no teste de similaridade cosseno!


In [19]:
!guardrails hub install hub://guardrails/pii --quiet

Installing hub://guardrails/pii...
ERROR:guardrails-cli:404
ERROR:guardrails-cli:Not Found
ERROR:guardrails-cli:Failed to install hub://guardrails/pii


In [20]:
@register_validator(name="guardrails/enrollment", data_type="string")
class PIIValidator(Validator):
    def __init__(self, on_match: Optional[str] = None, on_fail: Optional[Callable] = None):
        super().__init__(on_match=on_match, on_fail=on_fail)
        
    def validate(self, value: str, metadata: Dict = {}) -> ValidationResult:
        year = str(dt.datetime.now().year)[2:]
        
        enrollment = r"\b\d{1}[1-" + year[0] + r"]" + r"[0-" + year[1] + r"]" + r"[12]\d{5}\b"
        enrollment_pattern = Pattern(name="Enrollment_Pattern", regex=enrollment, score=0.6)
        
        enrollment_recognizer = PatternRecognizer(name="MATRICULA", patterns=[enrollment_pattern], supported_entity="Enrollment_Pattern", supported_language="en")
        
        analyzer = AnalyzerEngine()
        analyzer.registry.add_recognizer(enrollment_recognizer)
        
        results = analyzer.analyze(text=value, entities=["Enrollment_Pattern"], language="en")
        
        if results:
            for result in results:
                start, end = result.start, result.end
                value = value[:start] + "<MATRICULA>" + value[end:]
            return FailResult(error_message=value)
        
        return PassResult()



guardas_eureca = Guard().use(PIIValidator)

try:
    guardas_eureca.parse("A minha matricula é o seguinte: 221199999, busque informaões sobre ela").model_validate
except Exception as e:
    print(e)

Validation failed for field with errors: A minha matricula é o seguinte: <MATRICULA>, busque informaões sobre ela


In [21]:

@register_validator(name="guardrails/cpf", data_type="string")
class PIIValidatorCPF(Validator):
    def __init__(self, on_match: Optional[str] = None, on_fail: Optional[Callable] = None):
        super().__init__(on_match=on_match, on_fail=on_fail)
        
    def validate(self, value: str, metadata: Dict = {}) -> ValidationResult:
        year = str(dt.datetime.now().year)[2:]
        
        cpf_regex = r"\b\d{3}\.*\d{3}\.*\d{3}-*\d{2}\b"
        cpf_pattern = Pattern(name="CPF_Pattern", regex=cpf_regex, score=0.6)
        
        cpf_recognizer = PatternRecognizer(name="MATRICULA", patterns=[cpf_pattern], supported_entity="CPF_Pattern", supported_language="en")
        
        analyzer = AnalyzerEngine()
        analyzer.registry.add_recognizer(cpf_recognizer)
        
        results = analyzer.analyze(text=value, entities=["CPF_Pattern"], language="en")
        
        if results:
            for result in results:
                start, end = result.start, result.end
                value = value[:start] + "<CPF>" + value[end:]
            return FailResult(error_message=value)
        
        return PassResult()



guardas_eureca = Guard().use(PIIValidatorCPF)

try:
    result = guardas_eureca.parse("O meu CPF é o seguinte: 99999999999, busque informaões sobre ele").model_validate
except Exception as e:
    print(e)

Validation failed for field with errors: O meu CPF é o seguinte: <CPF>, busque informaões sobre ele


In [22]:
guarda_email = Guard().use(DetectPII(pii_entities="pii", on_fail="fix"))

text = """
Meus emails são demo@lol.com ou dominio@gmail.com ou dominio@hotmail.com e dominio@hotmail.com.br; 
e os meus números de telefones são esses (99) 999999999 ou (99)999999999 ou 99999999999 e 99999999999, 
busque no google.com.br ou uol.com. 
"""
output = guarda_email.parse(
    llm_output=text,
    metadata={"pii_entities": ["EMAIL_ADDRESS", "URL", "PHONE_NUMBER"]},
)

print(output.validated_output)


Meus emails são <EMAIL_ADDRESS> ou <EMAIL_ADDRESS> ou <EMAIL_ADDRESS> e <EMAIL_ADDRESS>; 
e os meus números de telefones são esses <PHONE_NUMBER> ou <PHONE_NUMBER> ou <PHONE_NUMBER> e <PHONE_NUMBER>, 
busque no <URL> ou <URL>. 



In [15]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline

@register_validator(name="hallucination_detector", data_type="string")
class HallucinationValidation(Validator):
    def __init__(
            self, 
            embedding_model: Optional[str] = None,
            entailment_model: Optional[str] = None,
            sources: Optional[List[str]] = None,
            **kwargs
        ):
        if embedding_model is None:
            embedding_model = 'all-MiniLM-L6-v2'
        self.embedding_model = SentenceTransformer(embedding_model)

        self.sources = sources
        
        if entailment_model is None:
            entailment_model = 'GuardrailsAI/finetuned_nli_provenance'
        self.nli_pipeline = pipeline("text-classification", model=entailment_model)

        super().__init__(**kwargs)

    def validate(
        self, value: str, metadata: Optional[Dict[str, str]] = None
    ) -> ValidationResult:
        sentences = self.split_sentences(value)

        relevant_sources = self.find_relevant_sources(sentences, self.sources)

        entailed_sentences = []
        hallucinated_sentences = []
        for sentence in sentences:
            is_entailed = self.check_entailment(sentence, relevant_sources)
            if not is_entailed:
                hallucinated_sentences.append(sentence)
            else:
                entailed_sentences.append(sentence)
        
        if len(hallucinated_sentences) > 0:
            return FailResult(
                error_message=f"Alucinação detectada: {hallucinated_sentences}",
            )
        
        return PassResult()

    def split_sentences(self, text: str) -> List[str]:
        return nltk.sent_tokenize(text)

    def find_relevant_sources(self, sentences: str, sources: List[str]) -> List[str]:
        source_embeds = self.embedding_model.encode(sources)
        sentence_embeds = self.embedding_model.encode(sentences)

        relevant_sources = []

        for sentence_idx in range(len(sentences)):
            sentence_embed = sentence_embeds[sentence_idx, :].reshape(1, -1)
            cos_similarities = np.sum(np.multiply(source_embeds, sentence_embed), axis=1)
            top_sources = np.argsort(cos_similarities)[::-1][:5]
            top_sources = [i for i in top_sources if cos_similarities[i] > 0.8]

            relevant_sources.extend([sources[i] for i in top_sources])

        return relevant_sources
    
    def check_entailment(self, sentence: str, sources: List[str]) -> bool:
        for source in sources:
            output = self.nli_pipeline({'text': source, 'text_pair': sentence})
            if output['label'] == 'entailment':
                return True
        return False

2025-01-20 16:56:30.276863: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-20 16:56:30.380682: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737402990.424061   34279 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737402990.436296   34279 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-20 16:56:30.539567: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [24]:
from transformers import AutoModel, AutoTokenizer

model_name = "GuardrailsAI/finetuned_nli_provenance"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

save_directory = "./saved_models/finetuned_nli_provenance"

tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

print(f"Modelo salvo em: {save_directory}")

Some weights of RobertaModel were not initialized from the model checkpoint at GuardrailsAI/finetuned_nli_provenance and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modelo salvo em: ./saved_models/finetuned_nli_provenance


In [25]:
save_directory = "./saved_models/finetuned_nli_provenance"

tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModel.from_pretrained(save_directory)

In [18]:
guard = Guard().use(
    HallucinationValidation(
        embedding_model='all-MiniLM-L6-v2',
        entailment_model='./saved_models/finetuned_nli_provenance',
        sources=[
        """
        ### Alunas (Feminino)

        - **Quantidade Total**: 183
        - **Estado Civil**
        - Solteiro: 175
        - Casado:3
        - **Nacionalidade**: 100% Brasileira
        - **Principais Estados**
        -PB:139
        - PE:17
        - **Idade**:
        - Média: 21.68 anos
        - Mínima: 17 anos
        - Máxima: 39 anos
        - **Política Afirmativa**:
        - Bon. estadual: 40
        - L2: 26
        - **Cor/Raça**:
          -Branca: 92
          -Parda: 83
        - **Renda Per Capita**:
        - Média: 6.78
        - **Tipo de Ensino Médio**
        - Somente escola pública: 93
        - Somente escola privada:89

        ### Alunos (Masculino)

        ⁃ **Quantidade Total**: 658
        - **Estado Civil**:
        ⁃ Solteiro: 641
        - Casado:6
        - **Nacionalidade**: Predominantemente Brasileira (657 brasileiros, 1 estrangeiro)
        - **Principais Estados**:
        -PB:498
        -PE: 47
        ⁃ **Idade**:
        ⁃ Média: 21.87 anos
        - Minima: 17 anos
        ⁃ Máxima: 43 anos
        - **Política Afirmativa**
        - Bon. estadual: 127
        - L2:71
        - **Cor/Raca**:
        ⁃ Branca: 337
        -Parda: 271
        - **Renda Per Capita**:
        - Média: 7.05
        - **Tipo de Ensino Médio**:
        ⁃ Somente escola pública: 329
        - Somente escola privada: 327

        Essas informações destacam a distribuição de gênero, origem
        geográfica, política afirmativa, e outros aspectos demograficos e
        educacionais dos alunos.
        """],
        on_fail=OnFailAction.EXCEPTION
    )
)

guard.validate("")

NameError: name 'Guard' is not defined

In [ ]:
from transformers import pipeline

CLASSIFIER = pipeline(
    "zero-shot-classification",
    model='facebook/bart-large-mnli',
    hypothesis_template="Verifique em qual tópico o texto acima melhor se adequa: {}.",
    multi_label=True,
)

dataset = [
    ("Quais são as disciplinas oferecidas no curso de Matemática", "disciplina"),
    ("Quantos cursos a universidade oferece", "curso"),
    ("Qual o calendário acadêmico deste semestre", "período"),
    ("Onde os alunos do curso de Engenharia são matriculados", "estudante"),
    ("Quantos professores tem em toda a universidade", "professor"),
    ("Quantos professores lecionam no curso de Psicologia", "professor"),
    ("Qual livro aborda o estudo de inteligência artificial", "livro"),
    ("Quais são os livros recomendados para o curso de Direito", "livro"),
    ("Quais as disciplinas do curso de Engenharia Civil", "disciplina"),
    ("Qual o período de férias da universidade", "período"),
    ("Quantos alunos estão matriculados no curso de ciencia da computacao", "estudante"),
    ("Quais são os livros que falam sobre estatística", "livro"),
]

class TopicTest:
    def __init__(self, dataset):
        self.dataset = dataset
        self.vocab_size = len(dataset)
        self.result = None
        self.topics = []
        self.test()

    def test(self):
        total_success = 0
        total_errors = 0
        errors = []
        self.topics = list(set([topic for _, topic in self.dataset]))
        
        for (input, output) in dataset:
            classified_output = CLASSIFIER(input, self.topics)
            classifier = classified_output['labels'][0]
            
            if classifier == output:
                total_success += 1
            else:
                errors.append({"sentence": input, "output_expected": output, "output_model": classifier, "score": classified_output['scores'][0]})
                total_errors += 1

        success_rate = total_success / self.vocab_size * 100
        errors_rate = total_errors / self.vocab_size * 100
        
        self.result = {
            "total_success": total_success,
            "total_errors": total_errors,
            "success_rate": success_rate, 
            "errors_rate": errors_rate,
            "errors_ocurred": errors
        }
    
    def explain_test(self):
        print(f'Obteve {self.result["total_success"]} acertos ({self.result["success_rate"]:.2f}%) e {self.result["total_errors"]} erro(s) ({self.result["errors_rate"]:.2f}%) de {tester.vocab_size} amostras')

tester = TopicTest(dataset=dataset)
print(tester.result)    
tester.explain_test() 

input = ""
classified_output = CLASSIFIER(input, ["disciplina", "curso", "período", "estudante", "professor", "livro"])

{'total_success': 11, 'total_errors': 1, 'success_rate': 91.66666666666666, 'errors_rate': 8.333333333333332, 'errors_ocurred': [{'sentence': 'Quantos professores lecionam no curso de Psicologia', 'expected_output': 'professor', 'model_predict': {'sentence': 'Quantos professores lecionam no curso de Psicologia', 'output_expected': 'professor', 'output_model': 'curso', 'score': 0.8182957172393799}}]}
Obteve 11 acertos (91.67%) e 1 erro(s) (8.33%) de 12 amostras
